## ETL Dados Antigos (2020-2022)

### 3.1. Justificativa Técnica e Abordagem Profissional

O pipeline de ETL dos Dados Antigos (2020-2022) foi desenvolvido com foco em **robustez e auditabilidade**, pilares essenciais da Engenharia de Dados.

| Desafio | Solução Profissional Implementada | Justificativa |
| :--- | :--- | :--- |
| **Inconsistência de Encoding** | Tentativa sequencial de `utf-8-sig`, `latin-1`, e `iso-8859-1`. | **Tolerância a Falhas:** Garante a extração de dados de fontes não padronizadas sem intervenção manual. |
| **Erros de Tipagem** | Limpeza robusta de caracteres (R$, vírgulas) antes da conversão para `float`. | **Integridade de Dados:** Previne a perda de dados e o erro de cálculo, crucial para métricas financeiras. |
| **Nomenclatura Inconsistente** | Uso de um `map_colunas` explícito. | **Manutenibilidade:** O mapeamento é necessário para lidar com **caracteres invisíveis** (Ex: `Descrição CATMAT` com Unicode `\xa0`) e garantir que a nomenclatura dos dados antigos seja **idêntica** ao formato dos dados novos, facilitando a integração futura no Data Warehouse. |
| **Rastreamento de Erros** | Uso de **`logging`** extenso em cada método e `try/except` com `traceback`. | **Auditabilidade:** Permite a identificação precisa do ponto de falha (qual arquivo, qual método) durante o processamento, uma necessidade crítica em ambientes de produção. |
| **Inversão de Colunas** | Lógica heurística baseada em contagem de letras/dígitos. | **Defesa contra Inconsistência:** Solução avançada para um problema comum em *datasets* legados, automatizando o que seria um trabalho manual e propenso a erros. |

### 3.2. Etapas do ETL (Fase de Preparação dos Dados)

A fase de **Preparação dos Dados** é a mais complexa no pipeline antigo, sendo orquestrada pelo método `_processar_arquivo`.

| Etapa | Método | Justificativa Técnica |
| :--- | :--- | :--- |
| **1. Padronização** | `_padronizar_colunas` | Uniformiza o nome das colunas para *snake\_case* e adiciona a coluna `ano_compra` para rastreamento. |
| **2. Correção de Estrutura** | `_corrigir_colunas_trocadas` | Realiza a correção heurística das colunas Nome/CNPJ trocadas, garantindo que o dado chegue à coluna correta. |
| **3. Tratamento de Tipos** | `_corrigir_tipos` | Aplica a limpeza robusta em campos numéricos (`qtd_itens_comprados`, `preco_unitario`), datas (`compra`, `insercao` com `dayfirst=True`) e padroniza campos categóricos (`generico` para SIM/NÃO, `tipo_compra` para ADMINISTRATIVA/JUDICIAL/INDEFINIDO). |
| **4. Preenchimento** | `_adicionar_colunas_vazias` | Insere colunas ausentes (`capacidade`, `unidade_medida`) com valores padrão (`0.0` e `'NA'`) para garantir a compatibilidade com o formato dos Dados Novos (2023+). |
| **5. Organização** | `_reordenar_colunas` | Garante que a ordem final das colunas seja idêntica ao *schema* esperado no Data Warehouse. |
| **6. Carga (Load)** | `processar_todos_antigos` | Consolida todos os DataFrames limpos (`pd.concat`) e salva o resultado final no formato CSV. |

### 3.3. Benefícios e Implicação para o TCC

O pipeline ETL Antigo atende aos objetivos de Modelagem e Avaliação do CRISP-DM, produzindo um conjunto de dados **confiável e pronto para o consumo**:

* **Modelagem Dimensional:** O processo garante que as chaves de ligação (CNPJs, `codigo_br`, `ano_compra`) estejam limpas e no formato **requerido** para a criação de um *Schema Star* (Tabela Fato).
* **Integridade de Dados:** O tratamento robusto de valores regionais e a correção de erros estruturais eliminam a necessidade de processamento manual, reduzindo a chance de erros humanos e garantindo a **confiabilidade** dos dados para a análise final do TCC.
* **Consolidação:** A junção de múltiplos arquivos CSV de forma automatizada permite a **análise de séries temporais** (2020 a 2022) com o mínimo de esforço de manutenção.

In [21]:
import os
import pandas as pd
import re
import traceback
from IPython.display import display

class ETLComprasAntigos:
    def __init__(self, pasta_base):
        self.pasta_base = pasta_base
        self.pasta_raw = os.path.join(self.pasta_base, "raw")
        self.pasta_processed = os.path.join(self.pasta_base, "processed")
        
        # Garante pastas
        os.makedirs(self.pasta_raw, exist_ok=True)
        os.makedirs(self.pasta_processed, exist_ok=True)

    # --- MÉTODOS DE TRANSFORMAÇÃO ---
    
    def _padronizar_colunas(self, df): 
        df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('__', '_')

        map_colunas = {
            'Ano': 'ano_compra',
            'Código_BR': 'codigo_br',
            'Descrição CATMAT': 'descricao_catmat',
            'Unidade_de_Fornecimento': 'unidade_fornecimento',
            'Genérico': 'generico',
            'Anvisa': 'anvisa',
            'Compra': 'compra',
            'Modalidade_da_Compra': 'modalidade_compra',
            'Inserção': 'insercao',
            'Tipo_Compra': 'tipo_compra',
            'Fabricante': 'cnpj_fabricante_temp',
            'CNPJ_Fabricante': 'fabricante_temp',
            'Fornecedor': 'cnpj_fornecedor_temp',
            'CNPJ_Fornecedor': 'fornecedor_temp',
            'Nome_Instituição': 'cnpj_instituicao_temp',
            'CNPJ_Instituição': 'nome_instituicao_temp',
            'Município_Instituição': 'municipio_instituicao',
            'UF': 'uf',
            'Qtd_Itens_Comprados': 'qtd_itens_comprados',
            'Preço_Unitário': 'preco_unitario',
        }
        
        df.rename(columns=map_colunas, inplace=True)
        return df

    def _corrigir_colunas_trocadas(self, df):
        def is_name(series):
            amostra = series.head(100).astype(str).str.replace(r'\\W', '', regex=True)
            letras = amostra.str.count(r'[a-zA-Z]').sum()
            digitos = amostra.str.count(r'[0-9]').sum()
            return letras > digitos * 0.5

        # Fornecedor
        if 'cnpj_fornecedor_temp' in df.columns and 'fornecedor_temp' in df.columns:
            if is_name(df['cnpj_fornecedor_temp']):
                df['fornecedor'] = df['cnpj_fornecedor_temp']
                df['cnpj_fornecedor'] = df['fornecedor_temp']
            else:
                df['fornecedor'] = df['fornecedor_temp']
                df['cnpj_fornecedor'] = df['cnpj_fornecedor_temp']
            df.drop(columns=['cnpj_fornecedor_temp', 'fornecedor_temp'], inplace=True)
        
        # Fabricante
        if 'cnpj_fabricante_temp' in df.columns and 'fabricante_temp' in df.columns:
            if is_name(df['cnpj_fabricante_temp']):
                df['fabricante'] = df['cnpj_fabricante_temp']
                df['cnpj_fabricante'] = df['fabricante_temp']
            else:
                df['fabricante'] = df['fabricante_temp']
                df['cnpj_fabricante'] = df['cnpj_fabricante_temp']
            df.drop(columns=['cnpj_fabricante_temp', 'fabricante_temp'], inplace=True)
            
        # Instituição
        if 'cnpj_instituicao_temp' in df.columns and 'nome_instituicao_temp' in df.columns:
            if is_name(df['cnpj_instituicao_temp']):
                df['nome_instituicao'] = df['cnpj_instituicao_temp']
                df['cnpj_instituicao'] = df['nome_instituicao_temp']
            else:
                df['nome_instituicao'] = df['nome_instituicao_temp']
                df['cnpj_instituicao'] = df['cnpj_instituicao_temp']
            df.drop(columns=['cnpj_instituicao_temp', 'nome_instituicao_temp'], inplace=True)
            
        return df

    def _corrigir_tipos(self, df):
        # 1. CORREÇÃO NUMÉRICA ROBUSTA
        colunas_numericas = ['qtd_itens_comprados', 'preco_unitario']
        for coluna in colunas_numericas:
            if coluna in df.columns:
                df[coluna] = df[coluna].astype(str).str.replace(r'[R$()+\s]', '', regex=True)
                df[coluna] = df[coluna].str.replace('.', '', regex=False)
                df[coluna] = df[coluna].str.replace(',', '.', regex=False)
                df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
                df[coluna] = df[coluna].fillna(0)

        # Recálculo de preco_total
        if 'preco_unitario' in df.columns and 'qtd_itens_comprados' in df.columns:
            df['preco_total'] = df['qtd_itens_comprados'] * df['preco_unitario']

        # 2. CORREÇÃO DE DATAS
        colunas_data = ['compra', 'insercao']
        for coluna in colunas_data:
            if coluna in df.columns:
                df[coluna] = pd.to_datetime(df[coluna], errors='coerce', dayfirst=True)
        
        # 3. CORREÇÃO DE TIPO DE COMPRA
        if 'tipo_compra' in df.columns:
            df['tipo_compra'] = (
                df['tipo_compra']
                .astype(str)
                .str.strip()
                .str.upper() 
                .replace({
                    'A': 'ADMINISTRATIVA',
                    'J': 'JUDICIAL',
                    'ADMINISTRATIVA': 'ADMINISTRATIVA',
                    'JUDICIAL': 'JUDICIAL',
                })
            )
            valores_validos = ['ADMINISTRATIVA', 'JUDICIAL']
            df['tipo_compra'] = df['tipo_compra'].apply(
                lambda x: 'INDEFINIDO' if x not in valores_validos else x
            )

        # 4. PADRONIZAÇÃO DO CÓDIGO BR
        if 'codigo_br' in df.columns:
            df['codigo_br'] = df['codigo_br'].astype(str).str.strip()
            df['codigo_br'] = (
                df['codigo_br']
                .str.upper()
                .str.replace(r'^BR0*', '', regex=True)
            )
            
            def limpar_codigo_br(codigo):
                if pd.isna(codigo):
                    return None
                codigo_limpo = re.sub(r'^BR', '', str(codigo).strip().upper()) 
                return codigo_limpo.strip() 
            
            df['codigo_br'] = df['codigo_br'].apply(limpar_codigo_br)

        # 5. CORREÇÃO CNPJ
        colunas_cnpj = ['cnpj_instituicao', 'cnpj_fornecedor', 'cnpj_fabricante']
        for coluna in colunas_cnpj:
            if coluna in df.columns:
                df[coluna] = df[coluna].astype(str).str.replace(r'\\D', '', regex=True).str.zfill(14)

        # 6. Flags (Generico)
        if 'generico' in df.columns:
            df['generico'] = (
                df['generico']
                .astype(str)
                .str.strip()
                .str.upper()
                .replace({'NÃO': 'NÃO', 'NAO': 'NÃO'})
                .fillna('NÃO')
            )
            df['generico'] = df['generico'].replace({'S': 'SIM', 'N': 'NÃO'})
            df['generico'] = df['generico'].apply(lambda x: 'NÃO' if x not in ['SIM', 'NÃO'] else x)

        # 7. Colunas de texto
        colunas_string = ['nome_instituicao', 'municipio_instituicao', 'uf',
                          'fornecedor', 'fabricante', 'descricao_catmat',
                          'unidade_fornecimento', 'modalidade_compra', 'tipo_compra', 'anvisa', 'codigo_br']
        for coluna in colunas_string:
            if coluna in df.columns:
                df[coluna] = df[coluna].astype(str).str.strip().replace('nan', '').replace('None', '')
        
        if 'anvisa' in df.columns:
            df['anvisa'] = df['anvisa'].str.replace(r'\\D', '', regex=True)

        return df
    
    def _adicionar_colunas_vazias(self, df):
        colunas_novas = {
            'capacidade': 0.0,
            'unidade_medida': 'NA',
        }
        for col, default in colunas_novas.items():
            if col not in df.columns:
                df[col] = default
        return df

    def _reordenar_colunas(self, df):
        colunas_finais = [
            'ano_compra', 'nome_instituicao', 'cnpj_instituicao', 'municipio_instituicao',
            'uf', 'compra', 'insercao', 'codigo_br', 'descricao_catmat',
            'unidade_fornecimento', 'generico', 'anvisa', 'modalidade_compra',
            'tipo_compra', 'capacidade', 'unidade_medida', 'cnpj_fornecedor',
            'fornecedor', 'cnpj_fabricante', 'fabricante', 'qtd_itens_comprados',
            'preco_unitario', 'preco_total'
        ]
        
        colunas_presentes = [col for col in colunas_finais if col in df.columns]
        for col in df.columns:
            if col not in colunas_presentes:
                colunas_presentes.append(col)
        
        df = df[colunas_presentes]
        return df

    def _processar_arquivo(self, df):
        df = self._padronizar_colunas(df)
        df = self._corrigir_colunas_trocadas(df)
        df = self._corrigir_tipos(df)
        df = self._adicionar_colunas_vazias(df)
        df = self._reordenar_colunas(df)
        return df
    
    def listar_arquivos_antigos(self):
        if not os.path.exists(self.pasta_raw):
            return []
            
        todos_arquivos = os.listdir(self.pasta_raw)
        arquivos_antigos = []
        ANOS_ANTIGOS = ['2020', '2021', '2022']
        
        for f in todos_arquivos:
            nome_base, ext = os.path.splitext(f)
            if ext.lower() == '.csv' and nome_base in ANOS_ANTIGOS:
                caminho = os.path.join(self.pasta_raw, f)
                arquivos_antigos.append(caminho)

        return arquivos_antigos

    def processar_todos_antigos(self):
        print(" Procurando arquivos na pasta 'raw'...")
        
        arquivos = self.listar_arquivos_antigos()
        
        if not arquivos:
            print(" Nenhum arquivo encontrado para processar")
            return None
            
        print(f" Arquivos encontrados: {len(arquivos)}")
        
        dfs = [] 
        anos_processados = []
        
        for arquivo_path in arquivos:
            nome_arquivo = os.path.basename(arquivo_path)
            print(f"\\n Processando: {nome_arquivo}")

            df = None
            encoding_tentativas = ['utf-8-sig', 'latin-1', 'iso-8859-1']

            for encoding in encoding_tentativas:
                try:
                    df = pd.read_csv(arquivo_path, sep=';', encoding=encoding, low_memory=False)
                    break
                except (UnicodeDecodeError, pd.errors.EmptyDataError):
                    continue
                except Exception as e:
                    print(f" Erro na leitura de {nome_arquivo}: {e}")
                    break

            if df is None or df.empty:
                print(f" Não foi possível ler o arquivo: {nome_arquivo}")
                continue 
                        
            try:
                df_processado = self._processar_arquivo(df)
                dfs.append(df_processado)
                ano = os.path.splitext(nome_arquivo)[0]
                anos_processados.append(ano)
                print(f" Processado: {nome_arquivo} ({len(df_processado):,} registros)")
            
            except Exception as e:
                print(f" Erro na transformação de {nome_arquivo}: {e}")

        if not dfs: 
            print(" Nenhum DataFrame foi processado com sucesso")
            return None

        print("\\n Consolidando DataFrames...")
        df_consolidado = pd.concat(dfs, ignore_index=True)
        
        # Salvamento
        anos_str = "_".join(sorted(set(anos_processados)))
        caminho_consolidado = os.path.join(self.pasta_processed, f"compras_antigos_consolidado_{anos_str}.csv")
        df_consolidado.to_csv(caminho_consolidado, index=False, encoding='utf-8-sig', sep=';')
        
        print(f" Consolidação completa!")
        print(f" Total de registros: {len(df_consolidado):,}")
        print(f" Anos processados: {', '.join(sorted(set(anos_processados)))}")
        print(f" Arquivo salvo: {caminho_consolidado}")
        
        return df_consolidado

In [22]:
def processar_anos_antigos():    
    etl_antigo = ETLComprasAntigos("../data") 
    return etl_antigo.processar_todos_antigos()

def gerar_relatorio_qualidade(df):
    print("\\n" + "="*60)
    print(" RELATÓRIO DE QUALIDADE DOS DADOS")
    print("="*60)
    
    metricas = {
        'Total de Registros': f"{len(df):,}",
        'Anos Processados': df['ano_compra'].nunique(),
        'Instituições Únicas': df['nome_instituicao'].nunique(),
        'Municípios Únicos': df['municipio_instituicao'].nunique(),
        'Fornecedores Únicos': df['fornecedor'].nunique(),
        'Fabricantes Únicos': df['fabricante'].nunique(),
        'Produtos Únicos (Código BR)': df['codigo_br'].nunique(),
        'Valor Total das Compras': f"R$ {df['preco_total'].sum():,.2f}",
        'Completude CNPJ': f"{(df['cnpj_instituicao'] != '').mean()*100:.1f}%",
        'Completude Preço Unitário': f"{df['preco_unitario'].notna().mean()*100:.1f}%",
    }
    
    for k, v in metricas.items():
        print(f"  {k:<30} {v}")
    
    print("\\n Distribuição por Ano:")
    dist_ano = df['ano_compra'].value_counts().sort_index()
    for ano, count in dist_ano.items():
        print(f"  {ano}: {count:,} registros ({count/len(df)*100:.1f}%)")
    
    print("\\n  Distribuição por Tipo de Compra:")
    dist_tipo = df['tipo_compra'].value_counts()
    for tipo, count in dist_tipo.items():
        print(f"  {tipo}: {count:,} registros ({count/len(df)*100:.1f}%)")
    
    print("\\n Distribuição por Genérico:")
    dist_generico = df['generico'].value_counts()
    for tipo, count in dist_generico.items():
        print(f"  {tipo}: {count:,} registros ({count/len(df)*100:.1f}%)")

# Execução principal
print("=" * 60)
print("ETL - COMPRAS PÚBLICAS ANOS ANTIGOS (2020-2022)")
print("=" * 60)

df_antigos = None

try:
    df_antigos = processar_anos_antigos()
    
    if df_antigos is not None:
        print(f"\\n🎉 PROCESSAMENTO CONCLUÍDO COM SUCESSO!")
        
        # Gerar relatório de qualidade
        gerar_relatorio_qualidade(df_antigos)
        
        # Exibir amostra dos dados
        print("\\n" + "="*60)
        print("AMOSTRA DOS DADOS PROCESSADOS")
        print("="*60)
        display(df_antigos.head(10))
        
    else:
        print("\\n O pipeline foi concluído, mas nenhum dado pôde ser consolidado.")
        
except Exception as e:
    print(f"\\n ERRO NA EXECUÇÃO: {e}")
    print(f"Detalhes: {traceback.format_exc()}")

ETL - COMPRAS PÚBLICAS ANOS ANTIGOS (2020-2022)
 Procurando arquivos na pasta 'raw'...
 Arquivos encontrados: 3
\n Processando: 2020.csv


C:\Users\debor\AppData\Local\Temp\ipykernel_16924\3431048272.py:106: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[coluna] = pd.to_datetime(df[coluna], errors='coerce', dayfirst=True)
C:\Users\debor\AppData\Local\Temp\ipykernel_16924\3431048272.py:106: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[coluna] = pd.to_datetime(df[coluna], errors='coerce', dayfirst=True)


 Processado: 2020.csv (71,227 registros)
\n Processando: 2021.csv
 Processado: 2021.csv (70,893 registros)
\n Processando: 2022.csv


C:\Users\debor\AppData\Local\Temp\ipykernel_16924\3431048272.py:106: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[coluna] = pd.to_datetime(df[coluna], errors='coerce', dayfirst=True)
C:\Users\debor\AppData\Local\Temp\ipykernel_16924\3431048272.py:106: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[coluna] = pd.to_datetime(df[coluna], errors='coerce', dayfirst=True)


 Processado: 2022.csv (69,028 registros)
\n Consolidando DataFrames...
 Consolidação completa!
 Total de registros: 211,148
 Anos processados: 2020, 2021, 2022
 Arquivo salvo: ../data\processed\compras_antigos_consolidado_2020_2021_2022.csv
\n🎉 PROCESSAMENTO CONCLUÍDO COM SUCESSO!
\n============================================================
 RELATÓRIO DE QUALIDADE DOS DADOS
  Total de Registros             211,148
  Anos Processados               3
  Instituições Únicas            435
  Municípios Únicos              592
  Fornecedores Únicos            3431
  Fabricantes Únicos             2972
  Produtos Únicos (Código BR)    10036
  Valor Total das Compras        R$ 38,476,976,431.04
  Completude CNPJ                100.0%
  Completude Preço Unitário      100.0%
\n Distribuição por Ano:
  2020: 71,227 registros (33.7%)
  2021: 70,893 registros (33.6%)
  2022: 69,028 registros (32.7%)
\n  Distribuição por Tipo de Compra:
  ADMINISTRATIVA: 197,102 registros (93.3%)
  JUDICIAL: 14,04

,ano_compra,nome_instituicao,cnpj_instituicao,municipio_instituicao,uf,compra,insercao,codigo_br,unidade_fornecimento,generico,...,capacidade,unidade_medida,cnpj_fornecedor,fornecedor,cnpj_fabricante,fabricante,qtd_itens_comprados,preco_unitario,preco_total,Descrição CATMAT
0,2020,FUNDO MUNICIPAL DE SAUDE,09102679000102,FERREIROS,PE,2020-11-20,2020-12-16,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,23680034000170,D.ARAUJO COMERCIAL EIRELI,61190096000192,EUROFARMA LABORATORIOS LTDA,60,3.780,226.8,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
1,2020,MUNICIPIO DE UBATUBA,46482857000196,UBATUBA,SP,2020-01-03,2020-03-31,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,67729178000491,COMERCIAL CIRURGICA RIOCLARENSE LTDA,08055634000153,IMEC-INDUSTRIA DE MEDICAMENTOS CUSTODIA LTDA -...,4500,1.770,7965.0,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
2,2020,FUNDO MUNICIPAL DE SAUDE DE ALIANCA,10759784000190,ALIANCA,PE,2020-01-06,2020-03-05,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,23232280000169,ZUCK PAPEIS LTDA,06628333000146,FARMACE - INDUSTRIA QUIMICO-FARMACEUTICA CEARE...,210,2.310,485.1,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
3,2020,FUNDO MUNICIPAL DE SAUDE DE CASCAVEL,09051532000122,CASCAVEL,PR,2020-01-25,2020-02-21,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,00874929000140,MED CENTER COMERCIAL LTDA,06628333000146,FARMACE - INDUSTRIA QUIMICO-FARMACEUTICA CEARE...,2000,1.875,3750.0,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
4,2020,FUNDO MUNICIPAL DE SAUDE,12306005000126,ITATUBA,PB,2020-01-29,2020-10-06,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,08674752000140,CIRURGICA MONTEBELLO LTDA,08055634000153,IMEC-INDUSTRIA DE MEDICAMENTOS CUSTODIA LTDA -...,240,2.130,511.2,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
5,2020,FUNDO MUNICIPAL DE SAUDE,10257028000162,TAILANDIA,PA,2020-01-31,2020-09-15,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,08393709000106,SILVA E DELGADO LTDA - ME,02456955000183,NATULAB LABORATORIO S.A,50,3.950,197.5,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
6,2020,FUNDO MUNICIPAL DE SAUDE DE FEIRA NOVA,11472134000121,FEIRA NOVA,PE,2020-01-31,2021-01-09,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,29868059000188,HOSPITALMED EIRELI,06628333000146,FARMACE - INDUSTRIA QUIMICO-FARMACEUTICA CEARE...,600,2.090,1254.0,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
7,2020,MUNICIPIO DE PRESIDENTE PRUDENTE,55356653000108,PRESIDENTE PRUDENTE,SP,2020-02-11,2020-05-12,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,67729178000491,COMERCIAL CIRURGICA RIOCLARENSE LTDA,08055634000153,IMEC-INDUSTRIA DE MEDICAMENTOS CUSTODIA LTDA -...,3300,1.720,5676.0,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
8,2020,FUNDO MUNICIPAL DE SAUDE DE CAMOCIM DE SAO FELIX,11870137000113,CAMOCIM DE SAO FELIX,PE,2020-02-17,2020-11-18,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,23232280000169,ZUCK PAPEIS LTDA,06628333000146,FARMACE - INDUSTRIA QUIMICO-FARMACEUTICA CEARE...,2500,1.890,4725.0,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
9,2020,MUNICIPIO DE FLORAI,75731000000160,FLORAI,PR,2020-02-18,2020-04-01,233632,"FRASCO 100,00 ML",NÃO,...,0.0,NA,01328535000159,CLASSMED - PRODUTOS HOSPITALARES LTDA - EPP,06628333000146,FARMACE - INDUSTRIA QUIMICO-FARMACEUTICA CEARE...,400,2.050,820.0,"PETROLATO, ASPECTO FÍSICO:LÍQUIDO, TIPO:LAXATI..."
